<H1 align = 'Center'> Spark Higher Level APIs </H1>
<H3> => Assignment 3 : Answer questions regarding Customers </H3>
<BR>

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder. \
appName("productCatalog"). \
config("spark.ui.port","0"). \
config("spark.sql.warehouse.dir","/user/itv012857/warehouse"). \
enableHiveSupport(). \
master("yarn"). \
getOrCreate()

In [5]:
! hadoop fs -head /public/trendytech/retail_db/customers/part-00000

1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521
2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126
3,Ann,Smith,XXXXXXXXX,XXXXXXXXX,3422 Blue Pioneer Bend,Caguas,PR,00725
4,Mary,Jones,XXXXXXXXX,XXXXXXXXX,8324 Little Common,San Marcos,CA,92069
5,Robert,Hudson,XXXXXXXXX,XXXXXXXXX,"10 Crystal River Mall ",Caguas,PR,00725
6,Mary,Smith,XXXXXXXXX,XXXXXXXXX,3151 Sleepy Quail Promenade,Passaic,NJ,07055
7,Melissa,Wilcox,XXXXXXXXX,XXXXXXXXX,9453 High Concession,Caguas,PR,00725
8,Megan,Smith,XXXXXXXXX,XXXXXXXXX,3047 Foggy Forest Plaza,Lawrence,MA,01841
9,Mary,Perez,XXXXXXXXX,XXXXXXXXX,3616 Quaking Street,Caguas,PR,00725
10,Melissa,Smith,XXXXXXXXX,XXXXXXXXX,8598 Harvest Beacon Plaza,Stafford,VA,22554
11,Mary,Huffman,XXXXXXXXX,XXXXXXXXX,3169 Stony Woods,Caguas,PR,00725
12,Christopher,Smith,XXXXXXXXX,XXXXXXXXX,5594 Jagged Embers By-pass,San Antonio,TX,78227
13,Mary,Baldwin,XXXXXXXXX,XXXXXXXXX,7922 Iron Oak Gardens,Caguas,PR,00725
14,Katherine

In [11]:
customer_schema = "cust_id int, cust_fname string, cust_lname string, email string, \
password string, street string, city string, state string, zipcode int"

In [12]:
print(customer_schema)

cust_id int, cust_fname string, cust_lname string, email string, password string, street string, city string, state string, zipcode int


In [13]:
cust_df = spark.read.csv("/public/trendytech/retail_db/customers/*", schema=customer_schema)

In [14]:
cust_df.show()

+-------+-----------+----------+---------+---------+--------------------+-------------+-----+-------+
|cust_id| cust_fname|cust_lname|    email| password|              street|         city|state|zipcode|
+-------+-----------+----------+---------+---------+--------------------+-------------+-----+-------+
|      1|    Richard| Hernandez|XXXXXXXXX|XXXXXXXXX|  6303 Heather Plaza|  Brownsville|   TX|  78521|
|      2|       Mary|   Barrett|XXXXXXXXX|XXXXXXXXX|9526 Noble Embers...|    Littleton|   CO|  80126|
|      3|        Ann|     Smith|XXXXXXXXX|XXXXXXXXX|3422 Blue Pioneer...|       Caguas|   PR|    725|
|      4|       Mary|     Jones|XXXXXXXXX|XXXXXXXXX|  8324 Little Common|   San Marcos|   CA|  92069|
|      5|     Robert|    Hudson|XXXXXXXXX|XXXXXXXXX|10 Crystal River ...|       Caguas|   PR|    725|
|      6|       Mary|     Smith|XXXXXXXXX|XXXXXXXXX|3151 Sleepy Quail...|      Passaic|   NJ|   7055|
|      7|    Melissa|    Wilcox|XXXXXXXXX|XXXXXXXXX|9453 High Concession|       Ca

In [17]:
cust_df.createOrReplaceTempView("cust_view")

<H3> 1. Find the total number of customers in each state.

In [54]:
cust_df.groupBy("state").count().sort("count", ascending = False).show()

+-----+-----+
|state|count|
+-----+-----+
|   PR| 4771|
|   CA| 2012|
|   NY|  775|
|   TX|  635|
|   IL|  523|
|   FL|  374|
|   OH|  276|
|   PA|  261|
|   MI|  254|
|   NJ|  219|
|   AZ|  213|
|   GA|  169|
|   MD|  164|
|   NC|  150|
|   VA|  136|
|   CO|  122|
|   OR|  119|
|   MA|  113|
|   TN|  104|
|   NV|  103|
+-----+-----+
only showing top 20 rows



In [19]:
spark.sql("SELECT state, count(1) FROM cust_view\
            GROUP BY state
            ORDER BY count(1) DESC")

state,count(1)
PR,4771
CA,2012
NY,775
TX,635
IL,523
FL,374
OH,276
PA,261
MI,254
NJ,219


<H3> 2. Find the top 5 most common last names among the customers.

In [21]:
cust_df.groupBy("cust_lname").count().sort("count", ascending = False).limit(5).show()

+----------+-----+
|cust_lname|count|
+----------+-----+
|     Smith| 4626|
|   Johnson|   76|
|  Williams|   69|
|     Jones|   65|
|     Brown|   62|
+----------+-----+



In [25]:
spark.sql("SELECT cust_lname, count(1) FROM cust_view \
            GROUP BY cust_lname \
            ORDER BY COUNT(1) DESC \
            LIMIT 5 \
          ")

cust_lname,count(1)
Smith,4626
Johnson,76
Williams,69
Jones,65
Brown,62


### 3. Check whether there are any customers whose zip codes are not valid.

In [28]:
cust_df.filter("length(zipcode) != 5").count()

5191

In [29]:
cust_df.count()

12435

In [41]:
spark.sql("SELECT COUNT(1) total_customers, \
            SUM(CASE WHEN length(zipcode) != 5 THEN 1 ELSE 0 END) invalid_zipcode, \
            SUM(CASE WHEN length(zipcode) != 5 THEN 1 ELSE 0 END)/COUNT(1)*100 as per \
            FROM cust_view")

total_customers,invalid_zipcode,per
12435,5191,41.74507438681142


<H4> Out of 12,435 customer, the zip code of 5,191 (41.75%) customers is invalid.

### 4. Count the number of customers who have valid zipcodes

In [42]:
cust_df.filter("length(zipcode) = 5").count()

7244

In [43]:
spark.sql("SELECT COUNT(1) total_customers, \
            SUM(CASE WHEN length(zipcode) = 5 THEN 1 ELSE 0 END) invalid_zipcode, \
            SUM(CASE WHEN length(zipcode) = 5 THEN 1 ELSE 0 END)/COUNT(1)*100 as per \
            FROM cust_view")

total_customers,invalid_zipcode,per
12435,7244,58.25492561318858


### 5. Find the number of customers from each city in the state of California(CA).

In [47]:
cust_df.filter("state = 'CA'").groupBy("city").count().show()

+-------------+-----+
|city         |count|
+-------------+-----+
|Corona       |14   |
|Pittsburg    |4    |
|Compton      |19   |
|Palo Alto    |6    |
|Hanford      |9    |
|Anaheim      |19   |
|Folsom       |6    |
|Napa         |8    |
|Temecula     |6    |
|Reseda       |6    |
|Encinitas    |17   |
|Oceanside    |24   |
|Cupertino    |9    |
|Oakland      |3    |
|Davis        |9    |
|Fontana      |18   |
|Mission Viejo|26   |
|Madera       |5    |
|Elk Grove    |10   |
|Bakersfield  |41   |
+-------------+-----+
only showing top 20 rows



In [50]:
spark.sql("SELECT city, count(1) FROM cust_view \
            WHERE state='CA' \
            GROUP BY city \
          ")

city,count(1)
Corona,14
Pittsburg,4
Compton,19
Palo Alto,6
Hanford,9
Anaheim,19
Folsom,6
Napa,8
Temecula,6
Reseda,6


In [55]:
spark.stop()